In [2]:
aux_file = 'Texas2k_series2025/Texas2k_series25_case1_summerpeak/Texas2k_series25_case1_summerpeak.AUX'

with open(aux_file, 'r') as f:
    for _ in range(50):
        print(f.readline().strip())


CustomFieldDescription (ObjectType,CustomType,CustomMaxOfType,CustomFieldCaption,CustomHeaderCaption,
CustomIncludeInDiff)
{
"Default" "Floating Point" 5 "Custom\Floating Point X" "Cust Float X" "YES"
"Default" "String" 5 "Custom\String X" "Cust String X" "YES"
"Default" "Integer" 5 "Custom\Integer X" "Cust Int X" "YES"
}
PWCaseInformation (Selected)
{
"NO "
<SUBDATA PWCaseHeader>
//Case Description
</SUBDATA>
}
Owner (Number,Name,DataMaintainerAssign)
{
1 "1" ""
}
Substation (Number,Name,IDExtra,Latitude,Longitude,DataMaintainerAssign,
DataMaintainerInheritBlock)
{
1 "ODESSA 2" "1"  31.841400 -102.315000 "" "NO "
2 "PRESIDIO 2" "2"  29.888046 -104.519139 "" "NO "
3 "O DONNELL 1" "3"  32.926389 -101.647778 "" "NO "
4 "BIG SPRING 5" "4"  32.240278 -101.473611 "" "NO "
5 "VAN HORN" "5"  31.093480 -104.624514 "" "NO "
6 "IRAAN 2" "6"  30.976638 -102.257753 "" "NO "
7 "PRESIDIO 1" "7"  29.600000 -104.300000 "" "NO "
8 "SANDERSON" "8"  30.028067 -101.970461 "" "NO "
9 "MONAHANS 2" "9"  31.4

In [3]:
def parse_substation_coords(aux_file):
    coords = {}
    with open(aux_file, 'r') as f:
        lines = f.readlines()
    
    substation_section = False
    for line in lines:
        line = line.strip()
        if line.startswith('Substation'):
            substation_section = True
            continue
        if substation_section:
            if line == '' or line.startswith('}'):
                break  # end of substation section
            # Line format example: 1 "ODESSA 2" "1"  31.841400 -102.315000 "" "NO "
            parts = line.split('"')
            if len(parts) >= 5:
                # parts[1] is bus name, parts[4] contains lat lon and extras
                bus_name = parts[1]
                rest = parts[4].strip().split()
                try:
                    lat = float(rest[0])
                    lon = float(rest[1])
                    coords[bus_name] = (lat, lon)
                except:
                    pass
    return coords

aux_file = 'Texas2k_series2025/Texas2k_series25_case1_summerpeak/Texas2k_series25_case1_summerpeak.AUX'
bus_coords = parse_substation_coords(aux_file)

print(f"Extracted {len(bus_coords)} bus coordinates")
print(list(bus_coords.items())[:10])


Extracted 1554 bus coordinates
[('ODESSA 2', (31.8414, -102.315)), ('PRESIDIO 2', (29.888046, -104.519139)), ('O DONNELL 1', (32.926389, -101.647778)), ('BIG SPRING 5', (32.240278, -101.473611)), ('VAN HORN', (31.09348, -104.624514)), ('IRAAN 2', (30.976638, -102.257753)), ('PRESIDIO 1', (29.6, -104.3)), ('SANDERSON', (30.028067, -101.970461)), ('MONAHANS 2', (31.484586, -103.161046)), ('GRANDFALLS', (31.165245, -103.125868))]


In [4]:
def parse_branch_data(raw_file):
    branches = []
    id_to_name = {}

    with open(raw_file, 'r') as f:
        lines = f.readlines()

    in_bus_section = False
    in_branch_section = False

    for line in lines:
        line = line.strip()
        if line.startswith("0 / END OF SYSTEM-WIDE DATA, BEGIN BUS DATA"):
            in_bus_section = True
            continue
        if line.startswith("0 / END OF BUS DATA, BEGIN LOAD DATA"):
            in_bus_section = False
        if line.startswith("0 / END OF GENERATOR DATA, BEGIN BRANCH DATA"):
            in_branch_section = True
            continue
        if line.startswith("0 / END OF BRANCH DATA"):
            in_branch_section = False

        if in_bus_section and not line.startswith("@") and line != "":
            parts = line.split(",")
            if len(parts) >= 2:
                try:
                    bus_id = int(parts[0].strip())
                    bus_name = parts[1].strip().strip("'")
                    id_to_name[bus_id] = bus_name
                except:
                    pass

        if in_branch_section and not line.startswith("@") and line != "":
            parts = line.split(",")
            if len(parts) >= 2:
                try:
                    from_bus = int(parts[0].strip())
                    to_bus = int(parts[1].strip())
                    branches.append((from_bus, to_bus))
                except:
                    pass

    return branches, id_to_name

# Run it
raw_file = 'Texas2k_series2025/Texas2k_series25_case1_summerpeak/Texas2k_series25_case1_summerpeak.RAW'
branches, id_to_name = parse_branch_data(raw_file)

print(f"Parsed {len(branches)} branches and {len(id_to_name)} bus names.")


Parsed 3993 branches and 2751 bus names.


In [6]:
# Normalize bus names to match formatting with AUX file
id_to_name_clean = {
    k: v.upper().replace('~', '').replace('  ', ' ').strip()
    for k, v in id_to_name.items()
}


In [9]:
# Step 0: Create voltage dictionary for each branch
voltage_dict = {}
for (from_id, to_id) in branches:
    from_name = id_to_name_clean.get(from_id, "").strip()
    to_name = id_to_name_clean.get(to_id, "").strip()

    # Find best match in bus_coords
    from_key = next((name for name in bus_coords if from_name.startswith(name)), None)
    to_key = next((name for name in bus_coords if to_name.startswith(name)), None)

    if from_key and to_key:
        # Estimate voltage as absolute difference in lat/lon (mock logic)
        # In practice, replace this with real voltage from RAW file if available
        lat1, lon1 = bus_coords[from_key]
        lat2, lon2 = bus_coords[to_key]
        est_voltage = round(abs(lat1 - lat2 + lon1 - lon2) * 1000, 1)  # Example dummy value
        voltage_dict[(from_id, to_id)] = est_voltage


In [10]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import numpy as np
import plotly.io as pio

pio.renderers.default = 'notebook_connected'

# Prepare voltage values
voltages = np.array([
    voltage_dict.get((a, b)) or voltage_dict.get((b, a)) or 0
    for (a, b) in branches
])

# Normalize voltages for color mapping
norm = mcolors.Normalize(vmin=voltages.min(), vmax=voltages.max())
colormap = plt.colormaps.get_cmap('viridis')
voltage_colors = [mcolors.to_hex(colormap(norm(v))) for v in voltages]

# Create base figure
fig = go.Figure()

# Add lines (branches) with voltage color
for i, (from_id, to_id) in enumerate(branches):
    from_name = id_to_name_clean.get(from_id, "")
    to_name = id_to_name_clean.get(to_id, "")
    
    from_match = next((coord for name, coord in bus_coords.items() if from_name.startswith(name)), None)
    to_match = next((coord for name, coord in bus_coords.items() if to_name.startswith(name)), None)
    
    if from_match and to_match:
        lat1, lon1 = from_match
        lat2, lon2 = to_match
        color = voltage_colors[i]
        v_val = voltages[i]

        fig.add_trace(go.Scattergeo(
            lon=[lon1, lon2],
            lat=[lat1, lat2],
            mode='lines',
            line=dict(width=1.5, color=color),
            hoverinfo='text',
            text=f"{from_name} → {to_name}<br>Voltage: {v_val} kV",
            showlegend=False
        ))

# Add substations (bus markers)
for name, (lat, lon) in bus_coords.items():
    fig.add_trace(go.Scattergeo(
        lon=[lon], lat=[lat],
        mode='markers',
        marker=dict(size=3, color='blue'),
        text=name,
        showlegend=False
    ))

# Add fake scatter to show color scale (colorbar)
fake_colorscale = [[i / 100, mcolors.to_hex(colormap(i / 100))] for i in range(101)]

fig.add_trace(go.Scattergeo(
    lon=[None],
    lat=[None],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        cmin=voltages.min(),
        cmax=voltages.max(),
        colorbar=dict(
            title=dict(text='Voltage (kV)', side='right'),
            ticks='outside',
            ticklen=5,
            len=0.75
        ),
        color=[voltages.min()],  # dummy value
        size=0.01,
    ),
    showlegend=False
))

# Configure map layout
fig.update_layout(
    title="Texas2k Interactive Grid — Voltage Colored Lines",
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showland=True,
        landcolor="rgb(243, 243, 243)",
        subunitcolor="rgb(217, 217, 217)",
        countrycolor="rgb(217, 217, 217)",
        showlakes=True,
        lakecolor="rgb(255, 255, 255)",
        center=dict(lat=31, lon=-99),
        lataxis_range=[25, 37],
        lonaxis_range=[-107, -92],
        resolution=50,
    ),
    height=800
)

fig.show()
fig.write_html("texas2k_voltage_map.html")



#### Above is an interactive map of Texas with all of its buses (blue dots) and lines (colored lines) that are graded by voltage capacity as shown in the legend on the right. Each bus can be hovered on to get the name/ location of the bus as well as its coordinates. 

__[here is a link to the interactive plot](https://htmlpreview.github.io/?https://raw.githubusercontent.com/ckwon06/C.Lee--2025-Summer-Internship-Materials/refs/heads/master/texas2k_voltage_map.html?token=GHSAT0AAAAAADFOB72LTR6LD7Q5WQOLWVCS2CUPNSA)__
